In [1]:
import numpy as np
import sys
import glob 
import os 
from pathlib import Path

sys.path.insert(0, 'C:\\Suite2P\\')
import suite2p

from suite2p.run_s2p import run_s2p

mouseName = 'GF218'

fast_diskpath = str(Path('D:\Foustoukos'))
save_path = str(Path('D:\Foustoukos' + '\\' +  mouseName))

print(type(fast_diskpath))

ops = {
        # file paths
        'look_one_level_down': False, # whether to look in all subfolders when searching for tiffs
        'fast_disk': fast_diskpath, # used to store temporary binary file, defaults to save_path0
        'delete_bin': False, # whether to delete binary file after processing
        'mesoscan': False, # for reading in scanimage mesoscope files
        'h5py': [], # take h5py as input (deactivates data_path)
        'h5py_key': 'data', #key in h5py where data array is stored
        'save_path0': save_path, # stores results, defaults to first item in data_path
        'subfolders': [],
        'diameter' : 15,
        # main settings
        'nplanes' : 1, # each tiff has these many planes in sequence
        'nchannels' : 1, # each tiff has these many channels per plane
        'functional_chan' : 1, # this channel is used to extract functional ROIs (1-based)
        'tau':  0.7, # this is the main parameter for deconvolution
        'fs': 30.,  # sampling rate (PER PLANE - e.g. if you have 12 planes then this should be around 2.5)
        'force_sktiff': False, # whether or not to use scikit-image for tiff reading
        # output settings
        'preclassify': 0.5, # apply classifier before signal extraction with probability 0.5 (turn off with value 0)
        'save_mat': True, # whether to save output as matlab files
        'combined': True, # combine multiple planes into a single result /single canvas for GUI
        'aspect': 1.0, # um/pixels in X / um/pixels in Y (for correct aspect ratio in GUI)
        # bidirectional phase offset
        'do_bidiphase': False,
        'bidiphase': 0,
        # registration settings
        'do_registration': 1, # whether to register data (2 forces re-registration)
        'keep_movie_raw': False,
        'nimg_init': 300, # subsampled frames for finding reference image
        'batch_size': 500, # number of frames per batch
        'maxregshift': 0.1, # max allowed registration shift, as a fraction of frame max(width and height)
        'align_by_chan' : 1, # when multi-channel, you can align by non-functional channel (1-based)
        'reg_tif': False, # whether to save registered tiffs
        'reg_tif_chan2': False, # whether to save channel 2 registered tiffs
        'subpixel' : 10, # precision of subpixel registration (1/subpixel steps)
        'smooth_sigma': 1.15, # ~1 good for 2P recordings, recommend >5 for 1P recordings
        'th_badframes': 1.0, # this parameter determines which frames to exclude when determining cropping - set it smaller to exclude more frames
        'pad_fft': False,
        # non rigid registration settings
        'nonrigid': True, # whether to use nonrigid registration
        'block_size': [128, 128], # block size to register (** keep this a multiple of 2 **)
        'snr_thresh': 1.2, # if any nonrigid block is below this threshold, it gets smoothed until above this threshold. 1.0 results in no smoothing
        'maxregshiftNR': 5, # maximum pixel shift allowed for nonrigid, relative to rigid
        # 1P settings
        '1Preg': False, # whether to perform high-pass filtering and tapering
        'spatial_hp': 50, # window for spatial high-pass filtering before registration
        'pre_smooth': 2, # whether to smooth before high-pass filtering before registration
        'spatial_taper': 50, # how much to ignore on edges (important for vignetted windows, for FFT padding do not set BELOW 3*ops['smooth_sigma'])
        # cell detection settings
        'roidetect': True, # whether or not to run ROI extraction
        'spatial_scale': 0, # 0: multi-scale; 1: 6 pixels, 2: 12 pixels, 3: 24 pixels, 4: 48 pixels
        'connected': True, # whether or not to keep ROIs fully connected (set to 0 for dendrites)
        'sparse_mode' : True,
        'nbinned': 10000, # max number of binned frames for cell detection
        'max_iterations': 30, # maximum number of iterations to do cell detection
        'threshold_scaling': 5., # adjust the automatically determined threshold by this scalar multiplier
        'max_overlap': 0.75, # cells with more overlap than this get removed during triage, before refinement
        'high_pass': 100, # running mean subtraction with window of size 'high_pass' (use low values for 1P)
        # ROI extraction parameters
        'inner_neuropil_radius': 2, # number of pixels to keep between ROI and neuropil donut
        'min_neuropil_pixels': 350, # minimum number of pixels in the neuropil
        'allow_overlap': False, # pixels that are overlapping are thrown out (False) or added to both ROIs (True)
        # channel 2 detection settings (stat[n]['chan2'], stat[n]['not_chan2'])
        'chan2_thres': 0.65, # minimum for detection of brightness on channel 2
        # deconvolution settings
        'baseline': 'maximin', # baselining mode (can also choose 'prctile')
        'win_baseline': 60., # window for maximin
        'sig_baseline': 10., # smoothing constant for gaussian filter
        'prctile_baseline': 8.,# optional (whether to use a percentile baseline)
        'neucoeff': .7,  # neuropil coefficient
        'xrange': np.array([0, 0]),
        'yrange': np.array([0, 0]),
      }



<class 'str'>


In [2]:
# provide an h5 path in 'h5py' or a tiff path in 'data_path'
# db overwrites any ops (allows for experiment specific settings)

datapath = Path('\\\\sv1files.epfl.ch\\Petersen-Lab\\data\\' + mouseName + '\\Recordings\\Imaging\\')
datapath = str(datapath)
imaging_days = glob.glob(os.path.join(datapath, "GF*"))
binpath = Path('D:\Foustoukos\BIN')
binpath = str(binpath)

for n,f in enumerate(imaging_days):
    imaging_days[n] = os.path.basename(f)
    
    
for n in range(len(imaging_days)):
    
    current_day_path = str(datapath) + '\\' + str(imaging_days[n]) +'\\'
    
    binpath = Path('D:\Foustoukos\BIN' + '\\' + mouseName + '\\' + str(imaging_days[n]))
    
    binpath = str(binpath)
    
    savepath = Path('D:\Foustoukos' + '\\' +  mouseName + '\\' + str(imaging_days[n]))
    
    savepath = str(savepath)
   
    print(current_day_path)
    
    db = {
          'h5py': [], # a single h5 file path
          'h5py_key': 'data',
          'look_one_level_down': False, # whether to look in ALL subfolders when searching for tiffs
          'data_path': [current_day_path], # a list of folders with tiffs 
                                                 # (or folder of folders with tiffs if look_one_level_down is True, or subfolders is not empty)

          'subfolders': [], # choose subfolders of 'data_path' to look in (optional)
          'fast_disk': binpath, # string which specifies where the binary file will be stored (should be an SSD)
          'save_path0' : savepath,
        }


    #print(db['data_path'])
    
    opsEnd=run_s2p(ops=ops,db=db)




\\sv1files.epfl.ch\Petersen-Lab\data\GF218\Recordings\Imaging\GF218_26112019\
{'h5py': [], 'h5py_key': 'data', 'look_one_level_down': False, 'data_path': ['\\\\sv1files.epfl.ch\\Petersen-Lab\\data\\GF218\\Recordings\\Imaging\\GF218_26112019\\'], 'subfolders': [], 'fast_disk': 'D:\\Foustoukos\\BIN\\GF218\\GF218_26112019', 'save_path0': 'D:\\Foustoukos\\GF218\\GF218_26112019'}
** Found 501 tifs - converting to binary **
time 2570.59 sec. Wrote tifs to binaries for 1 planes
>>>>>>>>>>>>>>>>>>>>> PLANE 0 <<<<<<<<<<<<<<<<<<<<<<
----------- REGISTRATION
Reference frame, 7.93 sec.
2500/145937 frames, 66.45 sec.
5000/145937 frames, 134.23 sec.
7500/145937 frames, 202.06 sec.
10000/145937 frames, 271.63 sec.
12500/145937 frames, 338.00 sec.
15000/145937 frames, 405.81 sec.
17500/145937 frames, 474.26 sec.
20000/145937 frames, 541.29 sec.
22500/145937 frames, 609.69 sec.
25000/145937 frames, 677.90 sec.
27500/145937 frames, 744.47 sec.
30000/145937 frames, 809.60 sec.
32500/145937 frames, 874.62

C:\Users\foustouk\.conda\envs\suite2p\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Masks made in 2.79 sec.
Extracted fluorescence from 204 ROIs in 145937 frames, 292.06 sec.
----------- Total 599.39 sec.
----------- SPIKE DECONVOLUTION
----------- Total 56.16 sec.
Plane 0 processed in 4503.79 sec (can open in GUI).
total = 7074.38 sec.
TOTAL RUNTIME 7075.57 sec
\\sv1files.epfl.ch\Petersen-Lab\data\GF218\Recordings\Imaging\GF218_28112019\
{'h5py': [], 'h5py_key': 'data', 'look_one_level_down': False, 'data_path': ['\\\\sv1files.epfl.ch\\Petersen-Lab\\data\\GF218\\Recordings\\Imaging\\GF218_28112019\\'], 'subfolders': [], 'fast_disk': 'D:\\Foustoukos\\BIN\\GF218\\GF218_28112019', 'save_path0': 'D:\\Foustoukos\\GF218\\GF218_28112019'}
** Found 547 tifs - converting to binary **
time 2916.15 sec. Wrote tifs to binaries for 1 planes
>>>>>>>>>>>>>>>>>>>>> PLANE 0 <<<<<<<<<<<<<<<<<<<<<<
----------- REGISTRATION
Reference frame, 8.24 sec.
2500/159170 frames, 65.02 sec.
5000/159170 frames, 133.47 sec.
7500/159170 frames, 200.47 sec.
10000/159170 frames, 267.09 sec.
12500/1591

C:\Users\foustouk\.conda\envs\suite2p\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Masks made in 2.88 sec.
Extracted fluorescence from 211 ROIs in 159170 frames, 321.10 sec.
----------- Total 666.00 sec.
----------- SPIKE DECONVOLUTION
----------- Total 61.56 sec.
Plane 0 processed in 4916.96 sec (can open in GUI).
total = 7833.11 sec.
TOTAL RUNTIME 7834.21 sec
\\sv1files.epfl.ch\Petersen-Lab\data\GF218\Recordings\Imaging\GF218_22112019\
{'h5py': [], 'h5py_key': 'data', 'look_one_level_down': False, 'data_path': ['\\\\sv1files.epfl.ch\\Petersen-Lab\\data\\GF218\\Recordings\\Imaging\\GF218_22112019\\'], 'subfolders': [], 'fast_disk': 'D:\\Foustoukos\\BIN\\GF218\\GF218_22112019', 'save_path0': 'D:\\Foustoukos\\GF218\\GF218_22112019'}
** Found 417 tifs - converting to binary **
time 1915.69 sec. Wrote tifs to binaries for 1 planes
>>>>>>>>>>>>>>>>>>>>> PLANE 0 <<<<<<<<<<<<<<<<<<<<<<
----------- REGISTRATION
Reference frame, 8.15 sec.
2500/117474 frames, 68.39 sec.
5000/117474 frames, 132.55 sec.
7500/117474 frames, 199.92 sec.
10000/117474 frames, 265.90 sec.
12500/1174

...

Masks made in 2.50 sec.
Extracted fluorescence from 192 ROIs in 102429 frames, 170.22 sec.
----------- Total 365.06 sec.
----------- SPIKE DECONVOLUTION
----------- Total 42.86 sec.
Plane 0 processed in 3096.04 sec (can open in GUI).
total = 4881.84 sec.
TOTAL RUNTIME 4882.59 sec
\\sv1files.epfl.ch\Petersen-Lab\data\GF218\Recordings\Imaging\GF218_24112019\
{'h5py': [], 'h5py_key': 'data', 'look_one_level_down': False, 'data_path': ['\\\\sv1files.epfl.ch\\Petersen-Lab\\data\\GF218\\Recordings\\Imaging\\GF218_24112019\\'], 'subfolders': [], 'fast_disk': 'D:\\Foustoukos\\BIN\\GF218\\GF218_24112019', 'save_path0': 'D:\\Foustoukos\\GF218\\GF218_24112019'}
** Found 437 tifs - converting to binary **
16000 frames of binary, time 225.22 sec.
time 2058.93 sec. Wrote tifs to binaries for 1 planes
>>>>>>>>>>>>>>>>>>>>> PLANE 0 <<<<<<<<<<<<<<<<<<<<<<
----------- REGISTRATION
Reference frame, 8.41 sec.
2500/126396 frames, 67.49 sec.
5000/126396 frames, 136.89 sec.
7500/126396 frames, 206.42 sec.
10

C:\Users\foustouk\.conda\envs\suite2p\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Masks made in 3.06 sec.
Extracted fluorescence from 226 ROIs in 126396 frames, 259.93 sec.
----------- Total 542.02 sec.
----------- SPIKE DECONVOLUTION
----------- Total 54.75 sec.
Plane 0 processed in 3984.55 sec (can open in GUI).
total = 6043.47 sec.
TOTAL RUNTIME 6044.56 sec
\\sv1files.epfl.ch\Petersen-Lab\data\GF218\Recordings\Imaging\GF218_25112019\
{'h5py': [], 'h5py_key': 'data', 'look_one_level_down': False, 'data_path': ['\\\\sv1files.epfl.ch\\Petersen-Lab\\data\\GF218\\Recordings\\Imaging\\GF218_25112019\\'], 'subfolders': [], 'fast_disk': 'D:\\Foustoukos\\BIN\\GF218\\GF218_25112019', 'save_path0': 'D:\\Foustoukos\\GF218\\GF218_25112019'}
** Found 548 tifs - converting to binary **
time 2524.52 sec. Wrote tifs to binaries for 1 planes
>>>>>>>>>>>>>>>>>>>>> PLANE 0 <<<<<<<<<<<<<<<<<<<<<<
----------- REGISTRATION
Reference frame, 8.32 sec.
2500/153168 frames, 71.17 sec.
5000/153168 frames, 139.68 sec.
7500/153168 frames, 208.06 sec.
10000/153168 frames, 278.14 sec.
12500/1531

C:\Users\foustouk\.conda\envs\suite2p\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Masks made in 2.77 sec.
Extracted fluorescence from 197 ROIs in 153168 frames, 311.36 sec.
----------- Total 646.84 sec.
----------- SPIKE DECONVOLUTION
----------- Total 58.24 sec.
Plane 0 processed in 4773.17 sec (can open in GUI).
total = 7297.69 sec.
TOTAL RUNTIME 7298.70 sec


In [8]:
datapath = Path('\\\\sv1files.epfl.ch\\Petersen-Lab\\data\\' + mouseName + '\\Recordings\\Imaging\\')
datapath = str(datapath)
imaging_days = glob.glob(os.path.join(datapath, "GF*"))
binpath = Path('D:\Foustoukos\BIN')
binpath = str(binpath)

for n,f in enumerate(imaging_days):
    imaging_days[n] = os.path.basename(f)
    
n = 1

current_day_path = str(datapath) + '\\' + str(imaging_days[n]) +'\\'

binpath = Path('D:\Foustoukos\BIN' + '\\' + mouseName + '\\' + str(imaging_days[n]))

binpath = str(binpath)

savepath = Path('D:\Foustoukos' + '\\' +  mouseName + '\\' + str(imaging_days[n]))

savepath = str(savepath)

print(current_day_path)

db = {
  'h5py': [], # a single h5 file path
  'h5py_key': 'data',
  'look_one_level_down': False, # whether to look in ALL subfolders when searching for tiffs
  'data_path': [current_day_path], # a list of folders with tiffs 
                                         # (or folder of folders with tiffs if look_one_level_down is True, or subfolders is not empty)

  'subfolders': [], # choose subfolders of 'data_path' to look in (optional)
  'fast_disk': binpath, # string which specifies where the binary file will be stored (should be an SSD)
  'save_path0' : savepath,
}


#print(db['data_path'])

opsEnd=run_s2p(ops=ops,db=db)


\\sv1files.epfl.ch\Petersen-Lab\data\GF187\Recordings\Imaging\GF187_24052019\
{'h5py': [], 'h5py_key': 'data', 'look_one_level_down': False, 'data_path': ['\\\\sv1files.epfl.ch\\Petersen-Lab\\data\\GF187\\Recordings\\Imaging\\GF187_24052019\\'], 'subfolders': [], 'fast_disk': 'D:\\Foustoukos\\BIN\\GF187\\GF187_24052019', 'save_path0': 'D:\\Foustoukos\\GF187\\GF187_24052019'}
FOUND OPS IN D:\Foustoukos\GF187\GF187_24052019\suite2p\plane0
FOUND BINARIES: D:\Foustoukos\BIN\GF187\GF187_24052019\suite2p\plane0\data.bin
SKIPPING REGISTRATION FOR ALL PLANES...
>>>>>>>>>>>>>>>>>>>>> PLANE 0 <<<<<<<<<<<<<<<<<<<<<<
----------- ROI DETECTION AND EXTRACTION
Binning movie in chunks of length 21
Binned movie [6305,497,502], 146.79 sec.
NOTE: estimated spatial scale ~12 pixels, time epochs 5.25, threshold 52.54 
0 ROIs, score=898.55
Found 800 ROIs, 333.88 sec
NOTE: applying classifier C:\Suite2P\suite2p\classifiers/classifier_user.npy
After classification with threshold 0.50, 368 ROIs remain
After re

C:\Users\foustouk\.conda\envs\suite2p\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Masks made in 5.60 sec.
Extracted fluorescence from 368 ROIs in 132412 frames, 331.32 sec.
----------- Total 680.86 sec.
----------- SPIKE DECONVOLUTION
----------- Total 80.68 sec.
Plane 0 processed in 765.91 sec (can open in GUI).
total = 766.45 sec.
TOTAL RUNTIME 767.36 sec
